# Amazon Augmented AI (Amazon A2I)

Amazon Augmented AI (Amazon A2I) makes it easy to build the workflows required for human review of ML predictions. Amazon A2I brings human review to all developers, removing the undifferentiated heavy lifting associated with building human review systems or managing large numbers of human reviewers. 

You can create your own workflows for ML models built on Amazon SageMaker or any other tools. Using Amazon A2I, you can allow human reviewers to step in when a model is unable to make a high confidence prediction or to audit its predictions on an on-going basis. 

Learn more here: https://aws.amazon.com/augmented-ai/

# Integrate human reviewers in Amazon Comprehend Text Classification

To incorporate Amazon A2I into your human review workflows, you need the following resources:

* **Workforce** to label your dataset. You can choose either the Amazon Mechanical Turk workforce, a vendor-managed workforce, or you can create your own private workforce for human reviews. Whichever workforce type you choose, Amazon Augmented AI takes care of sending tasks to workers.

* **Worker Task Template** to create a Human Task UI for the worker. The worker UI displays your input data, such as documents or images, and instructions to workers. It also provides interactive tools that the worker uses to complete your tasks. For more information, see https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-instructions-overview.html

* **Flow Definition** to create a Human Review Workflow.You use the flow definition to configure your human workforce and provide information about how to accomplish the human review task. You can create a flow definition in the Amazon Augmented AI console or with Amazon A2I APIs. To learn more about both of these options, see https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html

* **Human Loop** starts your human review workflow. When you use one of the built-in task types, the corresponding AWS service creates and starts a human loop on your behalf when the conditions specified in your flow definition are met or for each object if no conditions were specified. When a human loop is triggered, human review tasks are sent to the workers as specified in the flow definition.

When using a custom task type, as this notebook will show, you start a human loop using the Amazon Augmented AI Runtime API. When you call StartHumanLoop in your custom application, a task is sent to human reviewers.

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [ ]:
import io
import json
import uuid
import time
import boto3
import botocore

# Amazon Python SDK clients
sagemaker = boto3.client('sagemaker', region)
comprehend = boto3.client('comprehend', region)
a2i = boto3.client('sagemaker-a2i-runtime')
s3 = boto3.client('s3', region)

# Setup the S3 Output Location for the Workflow Results

In [ ]:
output_path = f's3://{bucket}/a2i-comprehend-star-rating-results'
print(output_path)

# Setup the Workforce and Workteam
A workforce is the group of workers that you have selected to label your dataset. When you use a private workforce, you also create work teams, a group of workers from your workforce that are assigned to Amazon Augmented AI human review tasks. You can have multiple work teams and can assign one or more work teams to each job.

To create a new Workforce and Workteam, navigate here:  https://console.aws.amazon.com/sagemaker/groundtruth#/labeling-workforces/create

<img src="img/augmented-create-workforce.png" width="80%" align="left">

--

<img src="img/augmented-create-workforce-confirmation-email.png" width="60%" align="left">

--

<img src="img/augmented-create-workforce-confirmation.png" width="80%" align="left">

--

# Update the WORKTEAM_ARN using the Private Team ARN above

In [ ]:
WORKTEAM_ARN = # 'arn:aws:sagemaker:X:Y:workteam/private-crowd/dsoaws'

Visit: https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-permissions-security.html to add the necessary permissions to your role

# Create the Human Task UI using a Worker Task Template

Create a human task UI resource, giving a UI template.  This template will be rendered to the human workers whenever human interaction is required.

Below we've provided a simple demo template that is compatible with AWS Comprehend's Detect Sentiment API.

For other pre-built UIs (70+), check: https://github.com/aws-samples/amazon-a2i-sample-task-uis

In [ ]:
template = r"""
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
    <crowd-classifier
      name="star_rating"
      categories="['1', '2', '3', '4', '5']"
      initial-value="{{ task.input.initialValue }}"
      header="Classify Reviews into Star Ratings Between 1 (Worst) and 5 (Best)"
    >
      <classification-target>
        {{ task.input.taskObject }}
      </classification-target>
      
      <full-instructions header="Instructions to Classify Reviews into Star Ratings Between 1 (Worst) and 5 (Best)">
        <p><strong>5</strong>: joy, excitement, delight</p>
        <p><strong>4</strong>: neutral-to-good</p>
        <p><strong>3</strong>: neither positive or negative, such as stating a fact</p>
        <p><strong>2</strong>: neutral-to-bad</p>
        <p><strong>1</strong>: anger, sarcasm, anxiety</p>        
      </full-instructions>

      <short-instructions>
       Classify Reviews into Star Ratings Between 1 (Worst) and 5 (Best)
      </short-instructions>
    </crowd-classifier>
</crowd-form>
"""

In [ ]:
# Task UI name - this value is unique per account and region. You can also provide your own value here.
taskUIName = 'ui-comprehend-' + str(uuid.uuid4()) 

# Create a Human Task UI resource.
humanTaskUiResponse = sagemaker.create_human_task_ui(
        HumanTaskUiName=taskUIName,
        UiTemplate={'Content': template})
humanTaskUiArn = humanTaskUiResponse['HumanTaskUiArn']
print(humanTaskUiArn)

# Create a Flow Definition

In this section, we're going to create a flow definition definition. Flow Definitions allow us to specify:

* The workforce that your tasks will be sent to.
* The instructions that your workforce will receive. This is called a worker task template.
* The configuration of your worker tasks, including the number of workers that receive a task and time limits to complete tasks.
* Where your output data will be stored.

This demo is going to use the API, but you can optionally create this workflow definition in the console as well. 

For more details and instructions, see: https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html.

In [ ]:
import uuid

# Flow definition name - this value is unique per account and region. You can also provide your own value here.
flowDefinitionName = 'fd-dsoaws-comprehend-' + str(uuid.uuid4()) 

create_workflow_definition_response = sagemaker.create_flow_definition(
        FlowDefinitionName= flowDefinitionName,
        RoleArn= role,
        HumanLoopConfig= {
            'WorkteamArn': WORKTEAM_ARN,
            'HumanTaskUiArn': humanTaskUiArn,
            'TaskCount': 1,
            'TaskDescription': 'Classify Reviews into Star Ratings Between 1 (Worst) and 5 (Best)',
            'TaskTitle': 'Classify Reviews into Star Ratings Between 1 (Worst) and 5 (Best)'
        },
        OutputConfig={
            'S3OutputPath' : output_path
        }
    )
flowDefinitionArn = create_workflow_definition_response['FlowDefinitionArn']

In [ ]:
# Describe flow definition - status should turn to "active"
for x in range(60):
    describeFlowDefinitionResponse = sagemaker.describe_flow_definition(FlowDefinitionName=flowDefinitionName)
    print(describeFlowDefinitionResponse['FlowDefinitionStatus'])
    if (describeFlowDefinitionResponse['FlowDefinitionStatus'] == 'Active'):
        print("Flow Definition is active")
        break
    time.sleep(2)

# Start a Human Loop When Comprehend Does Not Produce a Confident Prediction
Now that we have setup our Flow Definition, we are ready to call AWS Comprehend and start our human loops. In this tutorial, we are interested in starting a human loop if the confidence score returned by AWS Comprehend is less than a given threshold.  The human loop will engage our workforce for human review.

<img src="img/augmented-ai-workflow.png" width="70%" align="left">

In [ ]:
sample_reviews = [
                  'I enjoy this product', 
                  'I am unhappy with this product', 
                  'It is okay', 
                  'sometimes it works'
                 ]

# Retrieve the `comprehend_endpoint_arn` Deployed Previously

In [ ]:
%store -r comprehend_endpoint_arn

# Check the Confidence Score for Each Prediction by Comprehend
If < threshold, start the human loop.  You can integrate this type of logic into your application using the SDK.  In this case, we're using the Python SDK.

In [ ]:
human_loops_started = []

CONFIDENCE_SCORE_THRESHOLD = 0.50

for sample_review in sample_reviews:
    # Call the Comprehend Custom model that we trained earlier
    response = comprehend.classify_document(Text=sample_review, 
                                            EndpointArn=comprehend_endpoint_arn)

    star_rating = response['Classes'][0]['Name']
    confidence_score = response['Classes'][0]['Score']
    
    print(f'Processing sample_review: \"{sample_review}\"')
    
    # Our condition for when we want to engage a human for review
    if (confidence_score < CONFIDENCE_SCORE_THRESHOLD):
    
        humanLoopName = str(uuid.uuid4())
        inputContent = {
            'initialValue': star_rating,
            'taskObject': sample_review
        }
        start_loop_response = a2i.start_human_loop(
            HumanLoopName=humanLoopName,
            FlowDefinitionArn=flowDefinitionArn,
            HumanLoopInput={
                'InputContent': json.dumps(inputContent)
            }
        )

        human_loops_started.append(humanLoopName)

        print(f'Confidence score of {confidence_score} for star rating of {star_rating} is less than the threshold of {CONFIDENCE_SCORE_THRESHOLD}')
        print(f'*** ==> Starting human loop with name: {humanLoopName}  \n')
    else:
        print(f'Confidence score of {confidence_score} for star rating of {star_rating} is above threshold of {CONFIDENCE_SCORE_THRESHOLD}')
        print('No human loop created. \n')

# Check Status of Human Loop

In [ ]:
completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f'HumanLoop Name: {human_loop_name}')
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print('')
    
    if resp['HumanLoopStatus'] == 'Completed':
        completed_human_loops.append(resp)

# Wait For Workers to Complete Their Human Loop Tasks
Navigate to the link below and login with your email and password that you used when you setup the Private Workforce.

In [ ]:
workteamName = WORKTEAM_ARN[WORKTEAM_ARN.rfind('/') + 1:]
print('Navigate to the private worker portal and complete the human loop.')
print('Make sure you have invited yourself to the workteam and received the signup email.')
print('Note:  Check your spam filter if you have not received the email.')
print('')
print('https://' + sagemaker.describe_workteam(WorkteamName=workteamName)['Workteam']['SubDomain'])

# Start Labeling

<img src="img/augmented-comprehend-custom-start-working.png" width="80%" align="left">

# Select Label

<img src="img/augmented-comprehend-custom-select-label.png" width="80%" align="left">

# Loop is Completed

<img src="img/augmented-comprehend-custom-finished-task.png" width="80%" align="left">

# Verify the Human Loops are Completed

In [ ]:
completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f'HumanLoop Name: {human_loop_name}')
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print('')
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)

# View Human Labels  

Once the work is complete, Amazon A2I stores the results in the specified S3 bucket and sends a Cloudwatch Event.  Let's check the S3 contents.

In [ ]:
import re
import pprint

pp = pprint.PrettyPrinter(indent=4)

for resp in completed_human_loops:
    splitted_string = re.split('s3://' +  bucket + '/', resp['HumanLoopOutput']['OutputS3Uri'])
    output_bucket_key = splitted_string[1]

    response = s3.get_object(Bucket=bucket, Key=output_bucket_key)
    content = response['Body'].read()
    json_output = json.loads(content)
    pp.pprint(json_output)
    print('\n')